# Demo 4 - Manage and Monitor

In [1]:
from azureml.core import Workspace, Experiment, Run
ws = Workspace.get(name='BAUERWK',
subscription_id='52b56929-ee84-495c-91c3-a84dfacbc9d2',
resource_group='DP100'
)

In [2]:
ws.get_details()

{'id': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourceGroups/DP100/providers/Microsoft.MachineLearningServices/workspaces/BAUERWK',
 'name': 'BAUERWK',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': 'e45c443e-6661-4015-b3c2-7f2f6b9ddf4d',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-01-16T17:39:56.7000258+00:00',
 'containerRegistry': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourceGroups/DP100/providers/Microsoft.ContainerRegistry/registries/bauerwkd9e82596',
 'keyVault': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourcegroups/dp100/providers/microsoft.keyvault/vaults/bauerwk2103398596',
 'applicationInsights': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourcegroups/dp100/providers/microsoft.insights/components/bauerwk8495726486',
 'identityPrincipalId': '5e4de374-64a3-4a0d-b555-05478feb76aa',
 'identityTenantId': '60623c36-25e7-4dec

#### Registering Models to Azure Machine Learning Service

In [3]:
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl",
model_name = "sklearn_regression_model",
tags = {'area': "diabetes", 'type': "regression"},
description = "Ridge regression model to predict diabetes",
workspace = ws)

Registering model sklearn_regression_model


Querying the Registered Model

In [4]:
regression_models = Model.list(workspace=ws, tags=['area'])
for m in regression_models:
    print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

Name: sklearn_regression_model 	Version: 1 	Description: Ridge regression model to predict diabetes {'area': 'diabetes', 'type': 'regression'}
Name: sklearn_regression_model.pkl 	Version: 1 	Description: Ridge regression model to predict diabetes {'area': 'diabetes', 'type': 'regression'}


### Register Images Containing Machine Learning Models

The first step is to create the score.py file which instantiates the model and scores some sample data using it

In [5]:
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model
def init():
    global model
    model_path = Model.get_model_path('sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
# note you can pass in multiple rows for scoring
def run(raw_data):
        try:
            data = json.loads(raw_data)['data']
            data = numpy.array(data)
            result = model.predict(data)
# you can return any datatype if it is JSON-serializable 
            return result.tolist()
        except Exception as e:
            error = str(e)
        return error

The second step is to make sure the dependencies are included in the image

In [6]:
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

The third step is preparing the container image that will provide the environment to run the model

In [7]:
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime= "python",
execution_script="score.py",
conda_file="myenv.yml",
tags = {'area': "diabetes", 'type': "re-gression"},
description = "Image with ridge regression model")
image = Image.create(name = "myimage1",
# this is the model object
models = [model],
image_config = image_config,
workspace = ws)
image.wait_for_creation(show_output = True)

Creating image
Running...........................
Succeeded
Image creation operation finished for image myimage1:2, operation "Succeeded"


After the code is executed, you can view the images in the Azure portal in the Azure Machine Learning Service workspace

### Monitoring Models

The Azure Machine Learning Monitoring SDK is installed with the following command

In [8]:
pip install azureml-monitoring


The following command must be run outside of the IPython shell:

    $ pip install azureml-monitoring

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


Enable Data Collection

In [9]:
aks_config = AksWebservice.deploy_configuration(collect_model_data=True,enable_app_insights=True)

NameError: name 'AksWebservice' is not defined

### Collecting model data

First, import the Azure Machine Learning Monitoring module

In [10]:
from azureml.monitoring import ModelDataCollector

ModuleNotFoundError: No module named 'azureml.monitoring'

Second, add the following code to the init() function

In [ ]:
global inputs_dc, prediction_d
inputs_dc = ModelDataCollector("best_model", identifier="inputs", feature_
names=["feat1", "feat2", "feat3", "feat4", "feat5", "Feat6"])
prediction_dc = ModelDataCollector("best_model", identifier="predictions",feature_names=["prediction1", "prediction2"])
                                   

Third, add the collection function calls, lines below, to your run function

In [ ]:
inputs_dc.collect(data)
prediction_dc.collect(result)

Finally, update the myenv.yml file with the required modules needed to train the model

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])
myenv.add_pip_package("azureml-monitoring")
with open("myenv.yml","w") as f:
f.write(myenv.serialize_to_string())

The score.py file with the required calls to the collect() function appears as below

In [ ]:
%%writefile score.py
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model
from azureml.monitoring import ModelDataCollector
import time
def init():
global model
print ("model initialized" + time.strftime("%H:%M:%S"))
model_path = Model.get_model_path(model_name = 'sklearn_regression_model.
pkl')
# deserialize the model file back into a sklearn model
model = joblib.load(model_path)
global inputs_dc, prediction_dc
inputs_dc = ModelDataCollector(model_name="sklearn_regression_model",
identifier="inputs", feature_names=["feat1", "feat2"])
prediction_dc = ModelDataCollector("sklearn_regression_model", identifier="
predictions", feature_names=["prediction1", "prediction2"])
# note you can pass in multiple rows for scoring
def run(raw_data):
global inputs_dc, prediction_dc
try:
data = json.loads(raw_data)['data']
data = numpy.array(data)
    result = model.predict(data)
print ("saving input data" + time.strftime("%H:%M:%S"))
inputs_dc.collect(data) #this call is saving our input data into
our blob
prediction_dc.collect(result)#this call is saving our prediction
data into our blob
return result.tolist()
except Exception as e:
error = str(e)
print (error + time.strftime("%H:%M:%S"))
return error                               

### Where can I view the collected data?

The output of the monitoring service gets saved in the Azure BLOB storage account associated with your
Azure Machine Learning service workspace. The path of the file follows the following pattern

In [ ]:
/modeldata/<subscriptionid>/<resourcegroup>/<workspace>/<webservice>/<model>/<
version>/<identifier>/<year>/<month>/<day>/data.csv

An example would be

In [ ]:
/modeldata/1a2b3c4d-5e6f-7g8h-9i10-j11k12l13m14/myresourcegrp/myWorkspace/
aks-w-collv9/best_model/10/inputs/2018/12/31/data.csv

### Validate and analyze the collected data

Since the data gets added into Azure Blob storage, you can choose your favorite tool to run the analysis,
such as Power BI. We recommend completing the course Get started with Power BI3 to use Power BI to
analyze data on Azure BLOB storage

### Summary

Managing and monitoring machine learning pipelines can be an arduous task. In this module, you
learned how to use the Azure Machine Learning service model management and tracking feature to
automate this task. You registered a model and queried information about it. You registered the image
needed to support this mode. Then you installed the Azure ML Monitoring SDK and used it to enable
data collection, collect model training data, and query the collected data